In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sns.set_theme(style = 'whitegrid')

database = "../input/us-baby-names/database.sqlite"

In [ ]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)

tables

In [ ]:
names = pd.read_sql("""SELECT * FROM NationalNames
                        LIMIT 10;""", conn)
names

In [ ]:
states = pd.read_sql("""SELECT * FROM StateNames
                        LIMIT 10""", conn)
states

In [ ]:
# Colunas da tabela NationalNames
colunas_names = pd.read_sql("""PRAGMA table_info([NationalNames]);""", conn)
colunas_names

In [ ]:
# Colunas da tabela StateNames
colunas_states = pd.read_sql("""PRAGMA table_info([StateNames]);""", conn)
colunas_states

In [ ]:
# Quais os nomes mais susados entre 1980-2014 (Top 10)
cnt_names = pd.read_sql("""SELECT Name, SUM(Count) AS qtde FROM NationalNames
                            GROUP BY Name
                            ORDER BY SUM(Count) desc
                            LIMIT 10;""", conn)
plt.subplots(figsize=(6, 10))
sns.barplot(x = "qtde", y = 'Name', data = cnt_names, color="c")

Realizei a consulta do Top 10 nomes mais utilizados entre 1880 até 2014, não diferenciando o sexo dos bebês ou estado de nascimento.

Dificuldades: Tive dificuldade para arrumar a coluna do SUM(Count), para a contagem ir condizente com a quantidade das barras.
Há uma coluna na base de dados chamada Count, antes eu tinha feito a consulta fazendo um count pela quantidade de vezes que os nomes aparecem na lista, então tive que realizar uma soma usando a coluna count.

In [ ]:
# Qual o nome mais usado de cada sexo?
m_names = pd.read_sql("""SELECT Name, Gender, SUM(Count) FROM NationalNames
                              Where Gender = 'M'
                              GROUP BY name
                              ORDER BY SUM(Count) desc
                              LIMIT 1""", conn)
m_names

In [ ]:
f_names = pd.read_sql("""SELECT Name, Gender, SUM(Count) FROM NationalNames
                              WHERE Gender = 'F'
                              GROUP BY name
                              ORDER BY SUM(Count) desc
                              LIMIT 1""", conn)
f_names

Consulta sobre os nomes mais usados, de cada sexo dos bebês.
Consulta feita entre os anos 1880 até 2014, não diferenciando os estados onde cada nome foi utilizado.

Dificuldades: Não consegui consolidar as duas pesquisas em uma única consulta SQL. Tive que fazer duas consultas separadas.

In [ ]:
# Top 5 Geral - Masculino

top_male = pd.read_sql("""SELECT Name, Gender, SUM(Count) AS qtde FROM NationalNames
                          WHERE Gender = 'M'
                          GROUP BY name
                          ORDER BY SUM(Count) desc
                          Limit 5""", conn)

sns.barplot(x='qtde', y='Name', data=top_male, color='c')

Top 5 dos nomes utilizados nos bebês do sexo masculinos, entre os anos 1880 até 2014.
Não diferenciando o estado que cada bebê nasceu.

Dificuldades: A geração do gráfico, no eixo x a contagem não está correta.

In [ ]:
# Top 5 Geral - Feminino
top_female = pd.read_sql("""SELECT Name, Gender, SUM(Count) AS qtde FROM NationalNames
                            WHERE Gender = 'F'
                            GROUP BY Name
                            ORDER BY SUM(Count) desc
                            LIMIT 5""", conn)

sns.barplot(x='qtde', y='Name', data=top_female, color='c')

Top 5 dos nomes utilizados nos bebês do sexo femininos, entre os anos 1880 até 2014. Não diferenciando o estado que cada bebê nasceu.

Dificuldades: A geração do gráfico, no eixo x a contagem não está correta.

In [ ]:
# Top 5 no século XXI - Masculino
topm_xxi = pd.read_sql("""SELECT Name, Gender, SUM(Count) AS qtde FROM NationalNames
                         WHERE Year >= 2000 AND
                         Gender = 'M'
                         GROUP BY Name
                         ORDER BY SUM(Count) desc
                         LIMIT 5;""", conn)

sns.barplot(x='qtde', y='Name', data=topm_xxi, color='c')

Top 5 nomes mais utilizados no século XXI, dos bebês do sexo masculinho. Não diferenciando o estado que cada bebê nasceu.

Realizei essa pesquisa para saber se algum dos nomes dos bebês masculinos mais utilizados no top 5 geral apareceriam aqui neste top do século XXI.

Encontramos o nome Michael, que no top anterior estava em quarto lugar e neste consta em segundo.

Dificuldades: Usando o mesmo template para o gráfico de tops, o eixo x ficou diferente dos demais, como se a contagem estivesse funcionando.

In [ ]:
# Top 5 no século XXI - Feminino
topf_xxi = pd.read_sql("""SELECT Name, Gender, SUM(Count) AS qtde FROM NationalNames
                          WHERE Year >= 2000 AND
                          Gender = 'F'
                          GROUP BY Name
                          ORDER BY SUM(Count) desc
                          LIMIT 5;""", conn)

sns.barplot(x='qtde', y='Name', data=topf_xxi, color='c')

Top 5 nomes femininos mais utilizados no século XXI, não diferenciando os estados onde cara bebê nasceu.

O intuito da consulta é a mesma que foi dita sobre o top 5 nomes masculinos do século XXI, mas analisando o resultado, o top 5 nomes femininos anterior possuem a maior parte antes dos anos 2000.

Dificuldades: N/A

In [ ]:
# Qual estado mais utilizou o nome mais popular masculino (top 3)
topm_states = pd.read_sql("""SELECT State, SUM(Count) AS qtde FROM StateNames
                             WHERE Name = (
                             SELECT Name FROM NationalNames
                              WHERE Gender = 'M'
                              GROUP BY name
                              ORDER BY SUM(Count) desc
                              LIMIT 1)
                             GROUP BY State
                             ORDER BY SUM(Count) desc
                             LIMIT 3""", conn)

sns.barplot(x='qtde', y='State', data=topm_states, color='c')

Realizei o estudo dos top 3 estados americanos que usaram o nome masculino mais escolhido.
Como resultado temos os estados: 

* NY - New York
* PA - Pennsylvania
* CA - California

Dificuldades: Tive dificuldade para montar o SELECT dentro de outro SELECT, mas para usar o SUM(Count) apenas para a ordenação dos dados, sem trazê-lo como resultado.

In [ ]:
# Qual estado mais utilizou o nome mais popular feminino (top 3)
topf_states = pd.read_sql("""SELECT State, SUM(Count) AS qtde FROM StateNames
                             WHERE Name = (
                             SELECT Name FROM NationalNames
                              WHERE Gender = 'F'
                              GROUP BY name
                              ORDER BY SUM(Count) desc
                              LIMIT 1)
                             GROUP BY State
                             ORDER BY SUM(Count) desc
                             LIMIT 3""", conn)

sns.barplot(x='qtde', y='State', data=topf_states, color='c')

Realizei o estudo dos top 3 estados americanos que usaram o nome feminino mais escolhido.
Como resultado temos os estados:

* PA - Pennsylvania
* NY - New York
* TX - Texas

Dificuldades: as mesmas enfrendadas no top 3 anterior.

In [ ]:
# Qual estado com a maior variedade de nome (top 5)?
topv_states = pd.read_sql("""SELECT State, COUNT(DISTINCT(Name)) AS qtde FROM StateNames
                             GROUP BY State
                             ORDER BY COUNT(DISTINCT(Name)) desc
                             LIMIT 5;""", conn)

sns.barplot(x='qtde', y='State', data=topv_states, color='c')

Fiz a pesquisa para saber os 5 estados americamos que mais variaram na hora de escolher os nomes para os bebês.
Os estados são:
* CA - California
* TX - Texas
* NY - New York
* IL - Illinois
* FL - Florida

Dificuldades: Na hora da construção do COUNT(DISTINCT(Name)), para poder fazer a consulta e a contagem das variedades dos nomes de cara estado.